In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

from langchain_core.output_parsers import StrOutputParser

In [6]:
import os

In [7]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [12]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens = 500)

In [10]:
# Carregar PDF

pdf_path = os.path.join("..", "data", "os-sertoes.pdf")
loader = PyPDFLoader(pdf_path, extract_images=False)
pages = loader.load_and_split()

In [13]:
# Splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
parent_splitter = RecursiveCharacterTextSplitter(
  chunk_size=4000, 
  chunk_overlap=200, 
  length_function=len, 
  add_start_index=True,
  separators=["\n\n", "\n", ".", "!", "?", ",", " "]
)

In [14]:
# Storage

store = InMemoryStore()
vectorstore = Chroma(embedding_function=embeddings)

/var/folders/pp/byh0kn390x12x8dgh89r2d8r0000gn/T/ipykernel_50673/3164186608.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embeddings)


In [16]:
parent_document_retriever = ParentDocumentRetriever(
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    vectorstore=vectorstore,
    docstore=store
)
batch_size = 50
for i in range(0, len(pages), batch_size):
    batch_pages = pages[i:i + batch_size]
    parent_document_retriever.add_documents(batch_pages, ids=None)

In [17]:
parent_document_retriever.vectorstore.get()

{'ids': ['91e66b18-1a65-465d-962e-764db09b7c1d',
  'db1fd92c-07c9-4df5-aa36-e69e2908589a',
  '217afd10-f412-439a-8db9-1032a5b741ef',
  'bc233408-55d2-445d-b4b7-a991e9f3b7c5',
  'cf1b37c8-799b-4be3-aaf7-94891dfaaa9d',
  '6c788c05-e61b-4b2c-b80a-057d204b6574',
  'c7ad97e2-7818-4569-834c-a214747b5e65',
  '8dabb804-f8ca-4f25-84e4-b9242edccb71',
  'ee47391f-be5f-4a11-b1b0-778b887b84da',
  '43f3355a-80d9-41f9-a82c-1387d3aee9c4',
  'de78b67a-3520-454e-aa43-c5ce2341bc8e',
  '3e393f8f-50b8-4d3a-a1a4-8106e9d16dc5',
  '907fcb24-9d15-44dc-a6ef-2151827fc965',
  '205c0aec-6a98-4266-9b3f-98a6b8f40a72',
  '80ad40d0-706a-4487-b4de-da03f9305806',
  'fce9b42d-c8b8-4e8c-aef1-848d19930d75',
  'feae9f0e-3ee1-43fa-bc4d-efedbba360ca',
  '13919889-3e37-4e57-bb78-98f23a645ba1',
  '52c47f38-011e-41f7-9ea2-958ce47d71cb',
  'f306356a-21d7-4e24-a40f-35a9822772e5',
  '569f02e1-6fec-4f96-ba8b-8203e7d44051',
  '145ae493-2174-4dd7-9859-ebbdba9fa925',
  '6cde7e4b-472a-477e-9bc7-a354abd2fed1',
  '19956b68-d126-4fcb-9758-

In [22]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado

Contexto: {context}
    
Pergunta: {question}
"""

prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)
parser = StrOutputParser()
sequence = RunnableSequence(prompt | llm | parser)

In [23]:
def answer_question(question: str):
    context = parent_document_retriever.invoke(question)
    if not context:
        return "Desculpe, não consegui encontrar informações relevantes para a pergunta."

    response = sequence.invoke({"context": context, "question": question})
    if not response:
        return "Desculpe, não consegui gerar uma resposta para a pergunta."

    return response

In [24]:
for index, question in enumerate(questions):
  
    resposta = answer_question(question)
    
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha descreve o ambiente natural do sertão nordestino como uma região árida e selvagem, onde as condições geológicas e topográficas influenciam diretamente a vida dos habitantes. Ele destaca a influência das características genéticas do local, que reagem fortemente sobre os habitantes, contribuindo para o agravamento das condições de vida. A interação entre os agentes físicos e as condições locais resulta em um conflito perene, refletindo na significação mesológica do sertão. Euclides da Cunha ressalta que a região é pouco explorada pelos cientistas e que a vida no sertão é marcada por adversidades e dificuldades, tornando-se um local desconhecido e desafiador.'}
{'numero': 1, 'pergunta': 'Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essa